In [1]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



In [2]:

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

chunks = json.load(open("../data/chunks.json", "r", encoding="utf-8"))
index = faiss.read_index("../data/index.faiss")

In [3]:
def retrieve_context(query, k=5):
    
    q_emb = embedder.encode([query])
    D, I = index.search(np.array(q_emb), k)
    
    results = [chunks[i] for i in I[0]]
    
    return "\n\n---\n\n".join(results)

In [ ]:
query = "Ile dni urlopu przysługuje pracownikowi?"
context = retrieve_context(query)
print(context[:500])  # podgląd

In [ ]:


model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

def ask_llm(query, context):
    prompt = f"""
Odpowiadaj wyłącznie na podstawie dostarczonego kontekstu.
Jeśli w kontekście nie ma odpowiedzi – napisz to wprost.
Podawaj artykuły jeśli to możliwe.

Kontekst:
{context}

Pytanie:
{query}

Odpowiedź:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

answer = ask_llm(query, context)
print(answer)